In [2]:
### This is a Naive Bayes Classification Class
import nltk
from nltk import word_tokenize
import numpy as npy
import re
class MyNaiveBayes:
    def __init__(self):
    #initializing class variables
        self.classes = ["positive(+)", "negative(-)"]
        self.classNums = len(self.classes) 
        self.chanceOfOcc = {c : dict() for c in range(self.classNums) } 
        self.zvadzidza = [0 for i in range(self.classNums)]
        self.dictionaryOfWords = []
             
    
     #reads the files to be used
    def readDocs(self, document):
        retThus = []
        with open(document) as f:
            for line in f.readlines():
                this = line.split('\n')
                this = this[0].split('\t')
                revs = re.sub(r"[,/?!-()*&^%|'.,]","",this[0])
                bOw = word_tokenize(this[0].lower())
                label = int(this[1])
                retThus.append((self.mkBigram(bOw), label))
        print (retThus)
        return retThus

    def trainingDay(self, litium):
        classNumber = [0 for i in range(self.classNums)]
        lengthOf = len(litium)
        words = {c : dict() for c in range(self.classNums)}
        for book in litium:
            comment = book[0]
            col = book[-1]
            classNumber[col] += 1
            for izwi in comment:
                if izwi in words[col].keys():
                    words[col][izwi] += 1
                else:
                    words[col][izwi] = 1
                    
        for index in range(len(self.classes)):
            self.zvadzidza[index] = npy.log(classNumber[index]/lengthOf)
            self.dictionaryOfWords += list(words[index].keys())
        self.dictionaryOfWords = set(self.dictionaryOfWords)
        print (len(self.dictionaryOfWords))
        
        for index in range(len(self.classes)):
            for word in self.dictionaryOfWords:
                if word in words[index]:
                    self.chanceOfOcc[index][word] = npy.log((words[index][word]+1)/(sum(words[index].values())+len(self.dictionaryOfWords)))
                else:
                    self.chanceOfOcc[index][word] = npy.log((1)/(sum(words[index].values())+len(self.dictionaryOfWords)))

    #trains the classifier using input (type = text) referenced as books
    def trainor(self, books, test=False, split_ratio=0.30):
        litium = []
        for bk in books:
            for comment in self.readDocs(bk):
                litium.append(comment)
                
        if test:
            npy.random.shuffle(litium)
            splice = int(len(litium) * split_ratio)
            testSet = litium[:splice]
            trainSet = litium[splice:]
            self.trainingDay(trainSet)
            
            perfTest = self.testor(testSet)
            print ("You have", len(testSet)," test data")
            print ("Testing accuracy approximation: ",perfTest)
            perfTrain = self.testor(trainSet)
            print ("You have", len(trainSet)," train data")
            print ("Training accuracy approximation: ",perfTrain*100, "%")
        else:
            self.trainingDay(litium)

    
    #takes a token(s) as input and returns the numerical value of the class    
    def predictor(self, sentence):
        import operator
        tot = dict()
        for i in range(self.classNums):
            tot[i] = self.zvadzidza[i]
            for izwi in sentence:
                if izwi in self.dictionaryOfWords:
                    tot[i] = tot[i] + self.chanceOfOcc[i][izwi]
        return max(tot.items(), key=operator.itemgetter(1))[0]
    
    
    def testThisFile(self, zitaReFile):
        cols = []
        with open(zitaReFile) as f:
            for line in f.readlines():
                print(line,self.prediction(line))
                cols.append(self.prediction(line))
        
        with open('results.txt', 'w') as f:
            for cl in cols:
                print(line, self.prediction(line))
                f.write(str(cl)+"\n")    
        
                    
    #turns single words into bigrams for more efficient analysis and training           
    def mkBigram(self, words):
        data = []
        new = ['<s>'] + words + ['</s>']
        for i in range(len(new)-1):
            data.append(new[i]+'<>'+new[i+1])
        return data
    
    
    #tokenizes sentence, after which it uses the sentence to predict and assign the input to a class
    def prediction(self, text):
        sent = self.mkBigram(word_tokenize(text))
        return self.predictor(sent)
    
    def testor(self, data):
        allT = len(data)
        true = 0
        for book in data:
            reviews = book[0]
            column = book[-1]
            i = self.predictor(reviews)
            if (i == column): true += 1
        return true / allT

        
    #using already predefined labels, this function computes the accuracy of the mdel by checking the percentage of predictions that are akin to actual assignments
    def accuracy(self, output, actual):
        columnOutput = []
        actualColumns = []
        colRatio = 0
        
        with open(actual) as m:
            for line in m.readlines():
                mutupo = int(line)
                actualColumns.append(mutupo)
                
        with open(output) as m:
            for line in m.readlines():
                mutupo = int(line)
                columnOutput.append(mutupo)     
                
if __name__ == "__main__":
    classifier1 = MyNaiveBayes()
    classifier1.trainor(["./sentiment_labelled_sentences/amazon_cells_labelled.txt",
                  "./sentiment_labelled_sentences/imdb_labelled.txt",
                  "./sentiment_labelled_sentences/yelp_labelled.txt"],
                 test=True,
                 split_ratio=0.30)
    testing = input("Enter file name: ")
    classifier1.testThisFile(testing)    

[(['<s><>so', 'so<>there', 'there<>is', 'is<>no', 'no<>way', 'way<>for', 'for<>me', 'me<>to', 'to<>plug', 'plug<>it', 'it<>in', 'in<>here', 'here<>in', 'in<>the', 'the<>us', 'us<>unless', 'unless<>i', 'i<>go', 'go<>by', 'by<>a', 'a<>converter', 'converter<>.', '.<></s>'], 0), (['<s><>good', 'good<>case', 'case<>,', ',<>excellent', 'excellent<>value', 'value<>.', '.<></s>'], 1), (['<s><>great', 'great<>for', 'for<>the', 'the<>jawbone', 'jawbone<>.', '.<></s>'], 1), (['<s><>tied', 'tied<>to', 'to<>charger', 'charger<>for', 'for<>conversations', 'conversations<>lasting', 'lasting<>more', 'more<>than', 'than<>45', '45<>minutes.major', 'minutes.major<>problems', 'problems<>!', '!<>!', '!<></s>'], 0), (['<s><>the', 'the<>mic', 'mic<>is', 'is<>great', 'great<>.', '.<></s>'], 1), (['<s><>i', 'i<>have', 'have<>to', 'to<>jiggle', 'jiggle<>the', 'the<>plug', 'plug<>to', 'to<>get', 'get<>it', 'it<>to', 'to<>line', 'line<>up', 'up<>right', 'right<>to', 'to<>get', 'get<>decent', 'decent<>volume', 'v

[(['<s><>a', 'a<>very', 'very<>,', ',<>very', 'very<>,', ',<>very', 'very<>slow-moving', 'slow-moving<>,', ',<>aimless', 'aimless<>movie', 'movie<>about', 'about<>a', 'a<>distressed', 'distressed<>,', ',<>drifting', 'drifting<>young', 'young<>man', 'man<>.', '.<></s>'], 0), (['<s><>not', 'not<>sure', 'sure<>who', 'who<>was', 'was<>more', 'more<>lost', 'lost<>-', '-<>the', 'the<>flat', 'flat<>characters', 'characters<>or', 'or<>the', 'the<>audience', 'audience<>,', ',<>nearly', 'nearly<>half', 'half<>of', 'of<>whom', 'whom<>walked', 'walked<>out', 'out<>.', '.<></s>'], 0), (['<s><>attempting', 'attempting<>artiness', 'artiness<>with', 'with<>black', 'black<>&', '&<>white', 'white<>and', 'and<>clever', 'clever<>camera', 'camera<>angles', 'angles<>,', ',<>the', 'the<>movie', 'movie<>disappointed', 'disappointed<>-', '-<>became', 'became<>even', 'even<>more', 'more<>ridiculous', 'ridiculous<>-', '-<>as', 'as<>the', 'the<>acting', 'acting<>was', 'was<>poor', 'poor<>and', 'and<>the', 'the<>p

[(['<s><>wow', 'wow<>...', '...<>loved', 'loved<>this', 'this<>place', 'place<>.', '.<></s>'], 1), (['<s><>crust', 'crust<>is', 'is<>not', 'not<>good', 'good<>.', '.<></s>'], 0), (['<s><>not', 'not<>tasty', 'tasty<>and', 'and<>the', 'the<>texture', 'texture<>was', 'was<>just', 'just<>nasty', 'nasty<>.', '.<></s>'], 0), (['<s><>stopped', 'stopped<>by', 'by<>during', 'during<>the', 'the<>late', 'late<>may', 'may<>bank', 'bank<>holiday', 'holiday<>off', 'off<>rick', 'rick<>steve', 'steve<>recommendation', 'recommendation<>and', 'and<>loved', 'loved<>it', 'it<>.', '.<></s>'], 1), (['<s><>the', 'the<>selection', 'selection<>on', 'on<>the', 'the<>menu', 'menu<>was', 'was<>great', 'great<>and', 'and<>so', 'so<>were', 'were<>the', 'the<>prices', 'prices<>.', '.<></s>'], 1), (['<s><>now', 'now<>i', 'i<>am', 'am<>getting', 'getting<>angry', 'angry<>and', 'and<>i', 'i<>want', 'want<>my', 'my<>damn', 'damn<>pho', 'pho<>.', '.<></s>'], 0), (['<s><>honeslty', 'honeslty<>it', 'it<>did', "did<>n't", "

You have 900  test data
Testing accuracy approximation:  0.7844444444444445
You have 2100  train data
Training accuracy approximation:  99.57142857142857 %
Enter file name: this movie is boring


FileNotFoundError: [Errno 2] No such file or directory: 'this movie is boring'

In [ ]:
MyNaiveBayes()